In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\tom\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki

sf = open(datapath+'/Corpora/wiki/simple_20200601/simple_20200601_v2.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April is the fourth month of the year, and comes between March and May. 

 It is one of four months to have 30 days. 

 April always begins on the same day of week as July, and additionally, January in leap years. 

 April always ends on the same day of the week as December. 

 April's flowers are the Sweet Pea and Daisy. 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [5]:
simp = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/','simple_20200601_v2.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [6]:
## Not currently getting a paragraph = an article, we have additional blank lines coming through
i = 0
for p in simp.paras():
    if i > 5: break
    print(p)
    i +=1 

[['April', 'is', 'the', 'fourth', 'month', 'of', 'the', 'year,', 'and', 'comes', 'between', 'March', 'and', 'May.'], ['It', 'is', 'one', 'of', 'four', 'months', 'to', 'have', '30', 'days.'], ['April', 'always', 'begins', 'on', 'the', 'same', 'day', 'of', 'week', 'as', 'July,', 'and', 'additionally,', 'January', 'in', 'leap', 'years.'], ['April', 'always', 'ends', 'on', 'the', 'same', 'day', 'of', 'the', 'week', 'as', 'December.'], ["April's", 'flowers', 'are', 'the', 'Sweet', 'Pea', 'and', 'Daisy.'], ['Its', 'birthstone', 'is', 'the', 'diamond.'], ['The', 'meaning', 'of', 'the', 'diamond', 'is', 'innocence.']]
[['The', 'Month', 'Spring', 'flowers', 'in', 'April', 'in', 'the', 'Northern', 'Hemisphere.'], ['April', 'comes', 'between', 'March', 'and', 'May,', 'making', 'it', 'the', 'fourth', 'month', 'of', 'the', 'year.'], ['It', 'also', 'comes', 'first', 'in', 'the', 'year', 'out', 'of', 'the', 'four', 'months', 'that', 'have', '30', 'days,', 'as', 'June,', 'September', 'and', 'November'

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [19]:
for sent in sent_gen(simp):
    print(list(everygrams(sent,1,3)))
    for gram in everygrams(sent,1,3):
        print(len(gram))
    break

[('April',), ('is',), ('the',), ('fourth',), ('month',), ('of',), ('the',), ('year',), ('and',), ('comes',), ('between',), ('March',), ('and',), ('May',), ('April', 'is'), ('is', 'the'), ('the', 'fourth'), ('fourth', 'month'), ('month', 'of'), ('of', 'the'), ('the', 'year'), ('year', 'and'), ('and', 'comes'), ('comes', 'between'), ('between', 'March'), ('March', 'and'), ('and', 'May'), ('April', 'is', 'the'), ('is', 'the', 'fourth'), ('the', 'fourth', 'month'), ('fourth', 'month', 'of'), ('month', 'of', 'the'), ('of', 'the', 'year'), ('the', 'year', 'and'), ('year', 'and', 'comes'), ('and', 'comes', 'between'), ('comes', 'between', 'March'), ('between', 'March', 'and'), ('March', 'and', 'May')]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3


In [17]:
from nltk.util import ngrams, everygrams

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [8]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [9]:
scorer = NgramAssocMeasures.poisson_stirling

bi_cf = BigramCollocationFinder.from_words(word_gen(simp))
bi_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

tri_cf = TrigramCollocationFinder.from_words(word_gen(simp))
tri_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [10]:
print("\t", [" ".join(tup) for tup in bi_cf.nbest(BigramAssocMeasures.pmi, 15)])

	 ['$-125,050 $17,122,936', '$-178,219 $67,699,602', '$-390,187 $2,435,983', '$-4,228,696 $18,130,254', '$-666 $5,960,337', '$0.16 Timbit', '$1,032,603 $-390,187', '$1,078,876 $24,436,318', '$1,123,000,000 $11,686,214,000', '$1,188,341,317 $1,275,374,899', '$1,200,000 $425,488,741', '$1,275,374,899 $2,463,716,216', '$1,306,692 $29,103,686', '$1,403,380 $41,966', '$1,426,163 $1,403,380']


In [11]:
print("\t", [" ".join(tup) for tup in bi_cf.nbest(scorer, 15)])

	 ['$-125,050 $17,122,936', '$-178,219 $67,699,602', '$-390,187 $2,435,983', '$-4,228,696 $18,130,254', '$-666 $5,960,337', '$0.16 Timbit', '$1,032,603 $-390,187', '$1,078,876 $24,436,318', '$1,123,000,000 $11,686,214,000', '$1,188,341,317 $1,275,374,899', '$1,200,000 $425,488,741', '$1,275,374,899 $2,463,716,216', '$1,306,692 $29,103,686', '$1,403,380 $41,966', '$1,426,163 $1,403,380']


In [12]:
bi_dict = {}
tri_dict = {}

In [13]:
for bigram in bi_cf.score_ngrams(scorer):    
    bi_dict[bigram[0]] = [bigram[0], bi_cf.ngram_fd[bigram[0]], bigram[-1]]
    
bi_dict

{('$-125,050', '$17,122,936'): [('$-125,050', '$17,122,936'),
  1,
  -25.959351406439936],
 ('$-178,219', '$67,699,602'): [('$-178,219', '$67,699,602'),
  1,
  -25.959351406439936],
 ('$-390,187', '$2,435,983'): [('$-390,187', '$2,435,983'),
  1,
  -25.959351406439936],
 ('$-4,228,696', '$18,130,254'): [('$-4,228,696', '$18,130,254'),
  1,
  -25.959351406439936],
 ('$-666', '$5,960,337'): [('$-666', '$5,960,337'), 1, -25.959351406439936],
 ('$0.16', 'Timbit'): [('$0.16', 'Timbit'), 1, -25.959351406439936],
 ('$1,032,603', '$-390,187'): [('$1,032,603', '$-390,187'),
  1,
  -25.959351406439936],
 ('$1,078,876', '$24,436,318'): [('$1,078,876', '$24,436,318'),
  1,
  -25.959351406439936],
 ('$1,123,000,000', '$11,686,214,000'): [('$1,123,000,000', '$11,686,214,000'),
  1,
  -25.959351406439936],
 ('$1,188,341,317', '$1,275,374,899'): [('$1,188,341,317', '$1,275,374,899'),
  1,
  -25.959351406439936],
 ('$1,200,000', '$425,488,741'): [('$1,200,000', '$425,488,741'),
  1,
  -25.9593514064399

In [14]:
for trigram in tri_cf.score_ngrams(scorer):    
    tri_dict[trigram[0]] = [trigram[0], tri_cf.ngram_fd[trigram[0]], trigram[-1]]
    
tri_dict

{('$1,032,603',
  '$-390,187',
  '$2,435,983'): [('$1,032,603',
   '$-390,187',
   '$2,435,983'), 1, -25.959351406439936],
 ('$1,078,876',
  '$24,436,318',
  '$24,393,632'): [('$1,078,876',
   '$24,436,318',
   '$24,393,632'), 1, -25.959351406439936],
 ('$1,188,341,317',
  '$1,275,374,899',
  '$2,463,716,216'): [('$1,188,341,317',
   '$1,275,374,899',
   '$2,463,716,216'), 1, -25.959351406439936],
 ('$1,403,380', '$41,966', '$899,356'): [('$1,403,380', '$41,966', '$899,356'),
  1,
  -25.959351406439936],
 ('$1,426,163',
  '$1,403,380',
  '$41,966'): [('$1,426,163',
   '$1,403,380',
   '$41,966'), 1, -25.959351406439936],
 ('$1,427,133',
  '$1,766,819',
  '$2,403'): [('$1,427,133', '$1,766,819', '$2,403'), 1, -25.959351406439936],
 ('$1,510,759',
  '$36,915',
  '$2,035,609'): [('$1,510,759', '$36,915', '$2,035,609'),
  1,
  -25.959351406439936],
 ('$1,547,674',
  '$1,510,759',
  '$36,915'): [('$1,547,674',
   '$1,510,759',
   '$36,915'), 1, -25.959351406439936],
 ('$1,563', 'GDP_nominal

In [15]:
bigram_df = pd.DataFrame.from_dict(bi_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

trigram_df = pd.DataFrame.from_dict(tri_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

bigram_df

,ngram,freq,poisson
"($-125,050, $17,122,936)","($-125,050, $17,122,936)",1,-2.595935e+01
"($-178,219, $67,699,602)","($-178,219, $67,699,602)",1,-2.595935e+01
"($-390,187, $2,435,983)","($-390,187, $2,435,983)",1,-2.595935e+01
"($-4,228,696, $18,130,254)","($-4,228,696, $18,130,254)",1,-2.595935e+01
"($-666, $5,960,337)","($-666, $5,960,337)",1,-2.595935e+01
...,...,...,...
"(and, the)","(and, the)",50287,-2.520684e+06
"(to, the)","(to, the)",62517,-3.073387e+06
"(is, a)","(is, a)",83617,-3.944892e+06
"(in, the)","(in, the)",157322,-7.610418e+06


In [16]:
ngram_df = bigram_df.append(trigram_df).sort_values('poisson', ascending=False).reset_index(drop=True)

In [17]:
ngram_df['len'] = ngram_df.ngram.apply(len)

In [18]:
ngram_df

,ngram,freq,poisson,len
0,"($-125,050, $17,122,936)",1,-2.595935e+01,2
1,"(voleyu, spayana)",1,-2.595935e+01,2
2,"(vote3, Jeff5/10)",1,-2.595935e+01,2
3,"(vote11, Colby3/5)",1,-2.595935e+01,2
4,"(vorskane, Lalvar)",1,-2.595935e+01,2
...,...,...,...,...
23342685,"(and, the)",50287,-2.520684e+06,2
23342686,"(to, the)",62517,-3.073387e+06,2
23342687,"(is, a)",83617,-3.944892e+06,2
23342688,"(in, the)",157322,-7.610418e+06,2


In [19]:
# Pickle ngram_df - can re-use later with different approaches (ngram_df = pd.read_pickle() )
ngram_df.to_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_df.pkl')

In [20]:
len(simp.words())

31796513

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [21]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(simp, sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [22]:
stop

{'0',
 '1',
 '2',
 '3',
 '4',
 'A',
 'American',
 'He',
 'In',
 'It',
 'References',
 'The',
 'This',
 'a',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'born',
 'by',
 'can',
 'first',
 'for',
 'from',
 'had',
 'has',
 'have',
 'he',
 'his',
 'in',
 'is',
 'it',
 'not',
 'of',
 'on',
 'one',
 'or',
 'people',
 'that',
 'the',
 'their',
 'they',
 'to',
 'was',
 'were',
 'which',
 'with'}

In [23]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [24]:
min_freq = 20

In [25]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

ngram_df2

,ngram,freq,poisson,len
0,"(Ving, Rhames)",20,-6.056256e+02,2
1,"(Grădina, Zoologică)",20,-6.056256e+02,2
2,"(Gharb-Chrarda-Beni, Hssen)",20,-6.056256e+02,2
3,"(Karlovy, Vary)",20,-6.070334e+02,2
4,"(waystations, shuku-eki)",20,-6.070334e+02,2
...,...,...,...,...
215905,"(and, the)",50287,-2.520684e+06,2
215906,"(to, the)",62517,-3.073387e+06,2
215907,"(is, a)",83617,-3.944892e+06,2
215908,"(in, the)",157322,-7.610418e+06,2


In [26]:
eval_count = 150000

ngram_eval = ngram_df2[0:eval_count]

ngram_eval

,ngram,freq,poisson,len
0,"(Ving, Rhames)",20,-605.625590,2
1,"(Grădina, Zoologică)",20,-605.625590,2
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2
3,"(Karlovy, Vary)",20,-607.033377,2
4,"(waystations, shuku-eki)",20,-607.033377,2
...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3
149996,"(from, A)",58,-3071.612088,2
149997,"(1998, 8)",64,-3071.704678,2
149998,"(had, tested, positive)",55,-3071.728206,3


In [27]:
batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 15)

batches

{('Ving', 'Rhames'): 1,
 ('Grădina', 'Zoologică'): 1,
 ('Gharb-Chrarda-Beni', 'Hssen'): 1,
 ('Karlovy', 'Vary'): 1,
 ('waystations', 'shuku-eki'): 1,
 ('Edinson', 'Cavani'): 1,
 ('Jule', 'Styne'): 1,
 ('Flèche', 'Wallonne'): 1,
 ('maillot', 'jaune'): 1,
 ('Puffy', 'AmiYumi'): 1,
 ('Aspleniaceae', 'Asplenium'): 1,
 ('Sheela', 'Shrinivas'): 1,
 ('Caid', 'Essebsi'): 1,
 ('Bellas', 'Artes'): 1,
 ('Sonu', 'Nigam'): 1,
 ('Päijänne', 'Tavastia'): 1,
 ('Lemony', 'Snicket'): 1,
 ('Trisha', 'Yearwood'): 1,
 ('Pim', 'Fortuyn'): 1,
 ('Dany', 'Heatley'): 1,
 ('BNP', 'Paribas'): 1,
 ('Jomo', 'Kenyatta'): 1,
 ('Liev', 'Schreiber'): 1,
 ('Bertolt', 'Brecht'): 1,
 ('bel', 'canto'): 1,
 ('Dukla', 'Jihlava'): 1,
 ('Lieke', 'Martens'): 1,
 ('Primorje-Gorski', 'Kotar'): 1,
 ('Luchino', 'Visconti'): 1,
 ('mutually', 'intelligible'): 1,
 ('conventionally', 'delimit'): 1,
 ("Giant's", 'Causeway'): 1,
 ('Ruhollah', 'Khomeini'): 1,
 ('Jair', 'Bolsonaro'): 1,
 ('Tomáš', 'Berdych'): 1,
 ('Hawaiian–Emperor', 'seam

In [28]:
len(batches)

150000

In [29]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(Ving, Rhames)",1
1,"(Grădina, Zoologică)",1
2,"(Gharb-Chrarda-Beni, Hssen)",1
3,"(Karlovy, Vary)",1
4,"(waystations, shuku-eki)",1
...,...,...
149995,"(first, country, to)",2
149996,"(from, A)",-2
149997,"(1998, 8)",6
149998,"(had, tested, positive)",4


In [30]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Ving, Rhames)",20,-605.625590,2,1
1,"(Grădina, Zoologică)",20,-605.625590,2,1
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2,1
3,"(Karlovy, Vary)",20,-607.033377,2,1
4,"(waystations, shuku-eki)",20,-607.033377,2,1
...,...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3,2
149996,"(from, A)",58,-3071.612088,2,-2
149997,"(1998, 8)",64,-3071.704678,2,6
149998,"(had, tested, positive)",55,-3071.728206,3,4


In [31]:
ngram_eval.batch.value_counts()

 1     45301
 2     33312
 3     24686
 4     16668
 5     10517
 6      7502
 7      4240
 8      2702
 9      1905
 10     1051
-2       832
 11      450
-1       300
 12      247
 13      123
 14      107
 15       57
Name: batch, dtype: int64

In [32]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [33]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    sents_mwe = Sent_Seq(simp, batch_token_mwe)
    
    batch_model = Word2Vec(sents_mwe,
                             min_count = 20,  # 20 matches R&E on EN Wiki
                             size = 400,
                             workers = 8,
                             window = 5,
                             sg = 0,         # CBOW
                             sample = 10e-5, # Subsampling
                             negative = 10
                            )

    # Save model
    batch_model.save(datapath+'/Models/1 w2v/simple_batch{}.model'.format(bb+1))
    # Reload looks like    new_model = gensim.models.Word2Vec.load('/tmp/mymodel')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim', 'base_nearest', 'mwe_nearest'])

    for exp in batch_dfs[bb].ngram:
        # Combined token for MWE
        mwetoken = '+'.join(exp)
        
        # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
        sws = []
        # Component vectors
        cvs = []

        #  Neighbours in original & MWE-aware space
        oldn = []
        newn = []
      
        # List of individual word similarities (where present in the vocab)
        css = []
        
        # Empty array
        earr = np.empty(1000)
        earr[:] = np.nan
        
        # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
        #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
        if mwetoken in batch_model.wv.vocab:

            mwv = batch_model.wv[mwetoken]
            
            for w in exp:
                if w in batch_model.wv.vocab:
                    cvs.append(batch_model.wv[w])

                    oldn.append(batch_model.wv.most_similar(w, topn=5))

                    if w in stop:
                        sws.append(1)
                        css.append(np.nan)
                    else:
                        sws.append(0)
                        css.append(cosim(batch_model.wv[w], mwv ))

                # If component is absent from vocab
                else:
                    sws.append(-1)
                    cvs.append(earr)
                    css.append(np.nan)

                    oldn.append([])

            #  Mean cosim
            if min(sws) >= 0:
                cs = np.nanmean(css)
            else:
                cs = np.nan

            newn = batch_model.wv.most_similar(mwetoken, topn=5)
            
        # Combined token missing from vocab - mark with defaults
        else:
            sws = [-2]
            mwv = np.empty(400)
            mwv[:] = np.nan

            
        # Append to stats df
        statsf = statsf.append({
            'ngram'  : exp,
            'stopwords' : sws,
            'mwe_vector' : mwv,
            'component_vectors' : cvs,
            'component_cosims'  : css,
            'cosine_sim'  : cs,
            'base_nearest': oldn,
            'mwe_nearest' : newn,
        }, ignore_index=True)

    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    
    
batch_dfs

Wall time: 11h 57min 9s


{0:                              ngram  freq      poisson  len  batch  stopwords  \
 0                   (Ving, Rhames)    20  -605.625590    2      1   [-1, -1]   
 1             (Grădina, Zoologică)    20  -605.625590    2      1   [-1, -1]   
 2      (Gharb-Chrarda-Beni, Hssen)    20  -605.625590    2      1   [-1, -1]   
 3                  (Karlovy, Vary)    20  -607.033377    2      1   [-1, -1]   
 4         (waystations, shuku-eki)    20  -607.033377    2      1   [-1, -1]   
 ...                            ...   ...          ...  ...    ...        ...   
 45296             (the, South, of)    41 -3070.727239    3      1  [1, 0, 1]   
 45297                   (no, such)    64 -3070.763377    2      1     [0, 0]   
 45298      (very, important, part)    49 -3070.800963    3      1  [0, 0, 0]   
 45299        (a, children's, book)    49 -3071.464079    3      1  [1, 0, 0]   
 45300              (to, get, them)    46 -3071.464786    3      1  [1, 0, 0]   
 
                       

In [34]:
# Model inspection
batch_model.wv.similarity('April', 'August')
batch_model.wv.similarity('woman', 'man')
batch_model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
batch_model.wv.most_similar('relationships', topn=5)

[('relationship', 0.7281423807144165),
 ('behaviors', 0.7228208780288696),
 ('sexuality', 0.7120411396026611),
 ('struggles', 0.70902019739151),
 ('interactions', 0.7059136629104614)]

In [35]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Results/simple_output_001.csv', index=False)

In [36]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim,base_nearest,mwe_nearest
0,"(and, finally, the)",33,-2347.493609,3,4,"[1, 0, 1]","[0.13030347, 0.07994455, -0.082002535, 0.01398...","[[-0.40466067, 0.67198735, 0.6583337, 0.028511...","[nan, -0.16564158, nan]",-0.165642,"[[(which, 0.46104732155799866), (but, 0.460986...","[(It+serves+as, 0.715894341468811), (which+bel..."
1,"(was, born, here)",32,-2114.989504,3,7,"[1, 1, 0]","[0.06301035, -0.00019310784, 0.03390736, 0.043...","[[-0.22809601, 1.2524221, 0.4889323, 0.0955969...","[nan, nan, -0.16315266]",-0.163153,"[[(is, 0.6044977903366089), (became, 0.4967761...","[(né, 0.8337432742118835), (Swiss-born, 0.8315..."
2,"(and, most, recently)",39,-2537.673195,3,6,"[1, 0, 0]","[0.04911319, 0.0869296, -0.019135743, -0.06469...","[[-0.32700145, 0.02033321, 0.08639753, -0.2750...","[nan, -0.15468141, -0.16610241]",-0.160392,"[[(which, 0.48490220308303833), (but, 0.476355...","[(Videography, 0.7642322778701782), (Demos, 0...."
3,"(die, was)",37,-1847.535848,2,2,"[0, 1]","[-0.00041738697, -0.20349899, -0.1816036, 0.02...","[[-0.3992651, 0.18121262, 0.7204212, 0.2892096...","[-0.15876684, nan]",-0.158767,"[[(hurt, 0.6128476858139038), (dying, 0.605194...","[(two+terms+as, 0.6806028485298157), (Leahy, 0..."
4,"(is, both, the)",35,-2568.582045,3,5,"[1, 0, 1]","[0.08137203, 0.025552886, -0.053719494, -0.012...","[[0.22999515, 1.1195483, 0.13316496, -0.760339...","[nan, -0.1484074, nan]",-0.148407,"[[(was, 0.5818507671356201), (are, 0.480471730...","[(is+the+5th, 0.8106856346130371), (is+the+2nd..."
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,"(of, the, Verbandsgemeinde)",24,-1598.925917,3,9,"[1, 1, -1]","[0.086092666, 0.035355784, -0.09122365, -0.045...","[[-0.48178634, 1.2369498, 0.6219463, -0.015493...","[nan, nan, nan]",NaN,"[[(the, 0.5878567099571228), (country's, 0.462...","[(2town, 0.952147901058197), (of+the+Amt, 0.94..."
149996,"(a, star-shaped, medal)",32,-1810.953944,3,9,"[1, -1, 0]","[0.04534836, 0.2394793, -0.16489151, -0.127485...","[[-0.27947068, 0.868481, -0.35542133, -0.50611...","[nan, nan, 0.3035236]",NaN,"[[(another, 0.5744749307632446), (A, 0.5598182...","[(ribbon, 0.7906497716903687), (double-breaste..."
149997,"(Endowment, for, the)",41,-2639.302105,3,9,"[-1, 1, 1]","[0.12535019, 0.45642012, -0.06789097, 0.368823...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, nan, nan]",NaN,"[[], [(For, 0.4648495316505432), (supporting, ...","[(Humanities, 0.7652117013931274), (Academies,..."
149998,"(of, the, pre-Julian)",42,-2778.824226,3,9,"[1, 1, -1]","[0.1385226, -0.030978786, 0.21515429, 0.028620...","[[-0.48178634, 1.2369498, 0.6219463, -0.015493...","[nan, nan, nan]",NaN,"[[(the, 0.5878567099571228), (country's, 0.462...","[(Anno, 0.6430176496505737), (lunisolar, 0.633..."
